In [2]:
# import statements
import os
import shutil
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.offsetbox import OffsetImage, AnnotationBbox
from scipy.special import lambertw

matplotlib.rc_file_defaults()
plt.style.use(['bmh','/home/torres/Dropbox/python/styles/paper.mplstyle'])

import SPIutils as spi

from SPIworkflow.__init__ import *

# Import useful constants to be used in the code
from SPIworkflow.constants import *

In [11]:
df = pd.read_csv("./INPUT/SPI-sources_planets_MASTER.csv")
df2 = df[["star_name", "SP_TYPE", "d_star(pc)", "mass_star(m_sun)", "radius_star(r_sun)", "p_rot(days)", "bfield_star(gauss)",
          "planet_name", "p_orb(days)", "a(au)", "radius_planet(r_earth)", "mass_planet(m_earth)","ra(deg)", "dec(deg)"]]
#mask_Rpl = df2['radius_planet(r_earth)'] < 1.5
mask_d = df2['d_star(pc)'] < 15.0
mask_porb = df2['p_orb(days)'] < 10.0
#mask_Prot = df2['p_rot(days)'] < 20.0
mask_Bfield = df2['bfield_star(gauss)'] > 180.
mask_dec    = df2['dec(deg)'] > -54.0
#data = df2[mask_d & mask_Prot & mask_Bfield & mask_dec]
#data = df2[mask_Rpl & mask_d & mask_porb & mask_Bfield & mask_dec]
data = df2[mask_d & mask_porb & mask_Bfield & mask_dec]


#df2['freq_cycl(ghz)'] = df2.loc[:,('bfield_star(gauss)')]*2.8e-3
# Create new column
data['freq_cycl(ghz)'] = data['bfield_star(gauss)']*2.8e-3

#data = df2[mask_d & mask_Prot & mask_Porb & mask_a & mask_Bfield & mask_dec]
#data = data.drop_duplicates(subset=['star_name'])
data.reset_index(inplace=True)
data[:]

/tmp/ipykernel_13935/2958722864.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['freq_cycl(ghz)'] = data['bfield_star(gauss)']*2.8e-3


,index,star_name,SP_TYPE,d_star(pc),mass_star(m_sun),radius_star(r_sun),p_rot(days),bfield_star(gauss),planet_name,p_orb(days),a(au),radius_planet(r_earth),mass_planet(m_earth),ra(deg),dec(deg),freq_cycl(ghz)
0,0,CD Cet,M4.5Ve,8.609290,0.18,0.21,126.2,253,CD Cet b,2.290700,0.018500,1.439236,3.953432,48.345490,4.774818,0.7084
1,2,HD 285968,M2.5V,9.473043,0.51,0.51,40.7,350,GJ 176 b,8.774800,0.066000,1.718340,7.945000,70.732397,18.958165,0.9800
2,5,Wolf 1539,M2V,12.360389,0.40,0.40,NaN,204,GJ 179 c,3.479800,0.032000,1.519940,4.894120,73.023885,6.476553,0.5712
3,9,LP 656-038,M4.0Ve,5.375676,0.18,0.21,88.5,1135,GJ 3323 b,5.363600,0.032820,1.209451,2.033920,75.489275,-6.946214,3.1780
4,13,Luyten's Star,M3.5V,3.786059,0.30,0.30,93.5,219,GJ 273 c,4.723220,0.036000,1.050283,1.188572,111.852079,5.225789,0.6132
5,26,BD+48 1829,M1V,14.960944,0.58,0.59,40.5,195,Gl 378 b,3.822000,0.039435,2.341560,13.029800,150.590632,48.088802,0.5460
6,30,BD+01 2447,dM2,7.032918,0.45,0.45,32.7,273,GJ 393 b,7.026790,0.054020,1.156769,1.716120,157.231464,0.841001,0.7644
7,32,CN Leo,dM6,2.408597,0.14,0.15,2.7,3014,Wolf 359 c,2.686870,0.018000,1.425785,3.813600,164.120504,7.014723,8.4392
8,40,FI Vir,dM4,3.374875,0.20,0.22,163,279,Ross 128 b,9.865800,0.049600,1.097361,1.398320,176.934987,0.804557,0.7812
9,41,Wolf 433,M3.5Ve,14.241687,0.49,0.48,49.3,334,GJ 480 b,9.567000,0.068000,2.348285,13.188700,189.718504,11.696152,0.9352
